## Participation fault/subsections = Masterton example

In [1]:
import os
import pathlib
import json
from IPython.display import GeoJSON

import nzshm_model as nm
import solvis

from solvis.fault_system_solution_helper import (
    FaultSystemSolutionHelper,
    fault_participation_rate,
    section_participation_rate,
)

warning openquake module dependency not available, maybe you want to install
                with nzshm-model[openquake]


Running without `toshi` options


In [2]:
# load a single crustal Inversion solution from NSHM_v1.0.4
# sol = solvis.InversionSolution.from_archive("NZSHM22_ScaledInversionSolution-QXV0b21hdGlvblRhc2s6MTA3NzEy.zip") # Hikurangi
sol = solvis.InversionSolution.from_archive("NZSHM22_ScaledInversionSolution-QXV0b21hdGlvblRhc2s6MTEzMTM0.zip")  # Crustal 

In [3]:
hlpr = FaultSystemSolutionHelper(sol)

In [4]:
surfaces = sol.fault_surfaces()
FAULT_NAME = "Masterton"
aka_surface = surfaces[surfaces.ParentName==FAULT_NAME]

# A fault near Tuakau having 2 subsections
aka_surface # display the dataframe

,FaultID,FaultName,DipDeg,Rake,LowDepth,UpDepth,DipDir,AseismicSlipFactor,CouplingCoeff,ParentID,ParentName,geometry,Target Slip Rate,Target Slip Rate StdDev
1125,1125,"Masterton, Subsection 0",65.0,-160.0,12.88,0.0,164.0,0.0,1.0,257,Masterton,"POLYGON ((175.73310 -40.92830, 175.72130 -40.9...",0.000652,0.000577
1126,1126,"Masterton, Subsection 1",65.0,-160.0,12.88,0.0,164.0,0.0,1.0,257,Masterton,"POLYGON ((175.66992 -40.93846, 175.60882 -40.9...",0.000788,0.000592
1127,1127,"Masterton, Subsection 2",65.0,-160.0,12.88,0.0,164.0,0.0,1.0,257,Masterton,"POLYGON ((175.60882 -40.96057, 175.58770 -40.9...",0.000652,0.000530
1128,1128,"Masterton, Subsection 3",65.0,-160.0,12.88,0.0,164.0,0.0,1.0,257,Masterton,"POLYGON ((175.54348 -40.97084, 175.54080 -40.9...",0.000518,0.000468


In [5]:
# dispaly to the two fault sections for "Aka Aka" parent fablt
GeoJSON(json.loads(aka_surface.to_json()))

<IPython.display.GeoJSON object>

In [6]:
section_id = aka_surface.FaultID.unique().tolist()[-1]

In [7]:
# get the participation rate for a single (sub)section

# sec_id = 3 # the western Aka Aka 
rate = section_participation_rate(sol, section_id)
print(f"participation rate for section {section_id}: {rate} /yr")


participation rate for section 1128: 2.1576297513092868e-05 /yr


In [8]:
## Ruptures involving subsection 3

In [9]:
fids = hlpr.fault_names_as_ids([FAULT_NAME])
fids
# hlpr.ruptures_for_parent_fault_names([FAULT_NAME])
# len(hlpr.ruptures_for_faults(list(fids), True))
subsection_ids = hlpr.subsections_for_faults(list(fids))

df0 = sol.rupture_sections
df0 = df0.join(sol.rupture_rates.set_index("Rupture Index"), on='rupture', how='inner')[["Annual Rate", "rupture", "section"]]
df0 = df0[df0['Annual Rate'] >0]

ids = df0[df0['section'].isin(list(subsection_ids))]['rupture'].tolist()
set([int(id) for id in ids])

### code from ruptures_for_faults

{273738, 287882, 287883, 287886, 287935}

In [10]:
sol.rs_with_rupture_rates # this is returning a list of fault sections, joined to ruptures , including ruptures with 0 rates

,key_0,Rupture Index,Annual Rate,Magnitude,Average Rake (degrees),Area (m^2),Length (m),section
0,0,0,0.0,7.061465,110.000000,7.268884e+08,23211.794922,0.0
0,0,0,0.0,7.061465,110.000000,7.268884e+08,23211.794922,1.0
1,1,1,0.0,7.467223,110.000000,1.850232e+09,55766.039062,0.0
1,1,1,0.0,7.467223,110.000000,1.850232e+09,55766.039062,1.0
1,1,1,0.0,7.467223,110.000000,1.850232e+09,55766.039062,1302.0
...,...,...,...,...,...,...,...,...
411269,411269,411269,0.0,7.742744,161.768158,3.489376e+09,169210.187500,2206.0
411269,411269,411269,0.0,7.742744,161.768158,3.489376e+09,169210.187500,2207.0
411269,411269,411269,0.0,7.742744,161.768158,3.489376e+09,169210.187500,2202.0
411269,411269,411269,0.0,7.742744,161.768158,3.489376e+09,169210.187500,2204.0


In [11]:

aka_ruptures = hlpr.ruptures_for_parent_fault_names([FAULT_NAME])
aka_ruptures

        Rupture Index  Annual Rate
0                   0          0.0
1                   1          0.0
2                   2          0.0
3                   3          0.0
4                   4          0.0
...               ...          ...
411265         411265          0.0
411266         411266          0.0
411267         411267          0.0
411268         411268          0.0
411269         411269          0.0

[411270 rows x 2 columns]


{273738, 287882, 287883, 287886, 287935}

In [12]:
# rates for the Aka Ruptures
df = sol.ruptures_with_rupture_rates[["Rupture Index", "Annual Rate"]]
df1 = df[df["Rupture Index"].isin(list(aka_ruptures))]
df1
# df1[df1["Annual Rate"] > 0]

,Rupture Index,Annual Rate
273738,273738,0.000022
287882,287882,0.000062
287883,287883,0.000062
287886,287886,0.000008
287935,287935,0.000002


In [13]:
hlpr.ruptures_for_parent_fault_names([FAULT_NAME])

        Rupture Index  Annual Rate
0                   0          0.0
1                   1          0.0
2                   2          0.0
3                   3          0.0
4                   4          0.0
...               ...          ...
411265         411265          0.0
411266         411266          0.0
411267         411267          0.0
411268         411268          0.0
411269         411269          0.0

[411270 rows x 2 columns]


{273738, 287882, 287883, 287886, 287935}

Note that only one of the 5 ruptures has annual Rate. And this rate equals the ***participation rate for fault Aka Aka:*** **0.000049**

In [14]:
# let's inspect that rupture

In [15]:
sects = hlpr.subsections_for_ruptures([9])
rupt9_surface = surfaces[surfaces.FaultID.isin(sects)]
rupt9_surface

,FaultID,FaultName,DipDeg,Rake,LowDepth,UpDepth,DipDir,AseismicSlipFactor,CouplingCoeff,ParentID,ParentName,geometry,Target Slip Rate,Target Slip Rate StdDev
3,3,"Aka Aka, Subsection 0",65.0,-90.0,18.56,0.0,160.7,0.0,1.0,1,Aka Aka,"POLYGON ((174.82840 -37.26050, 174.84940 -37.2...",0.000266,0.000065
4,4,"Aka Aka, Subsection 1",65.0,-90.0,18.56,0.0,160.7,0.0,1.0,1,Aka Aka,"POLYGON ((174.87485 -37.25035, 174.88430 -37.2...",0.000266,0.000065
920,920,"Kerepehi Offshore, Subsection 0",60.0,-110.0,17.84,0.0,259.9,0.0,1.0,201,Kerepehi Offshore,"POLYGON ((175.45010 -37.21200, 175.43309 -37.1...",0.000567,0.000210
921,921,"Kerepehi Offshore, Subsection 1",60.0,-110.0,17.84,0.0,259.9,0.0,1.0,201,Kerepehi Offshore,"POLYGON ((175.43309 -37.14560, 175.41610 -37.0...",0.000693,0.000210
1102,1102,"Mangatangi, Subsection 0",65.0,-90.0,17.84,0.0,130.0,0.0,1.0,249,Mangatangi,"POLYGON ((175.14500 -37.21930, 175.16290 -37.2...",0.000986,0.000284
1103,1103,"Mangatangi, Subsection 1",65.0,-90.0,17.84,0.0,130.0,0.0,1.0,249,Mangatangi,"POLYGON ((175.19411 -37.15446, 175.20100 -37.1...",0.000418,0.000226
1574,1574,"Pokeno, Subsection 0",65.0,-90.0,18.08,0.0,157.5,0.0,1.0,365,Pokeno,"POLYGON ((175.13130 -37.19540, 175.11120 -37.1...",0.000423,0.000181
1575,1575,"Pokeno, Subsection 1",65.0,-90.0,18.08,0.0,157.5,0.0,1.0,365,Pokeno,"POLYGON ((175.02669 -37.22123, 175.01350 -37.2...",0.000441,0.000257


In [16]:
GeoJSON(json.loads(rupt9_surface.to_json()))

<IPython.display.GeoJSON object>

In [21]:
rate = fault_participation_rate(sol, FAULT_NAME)
print()
print(f"participation rate for fault '{FAULT_NAME}' = {rate} events/yr")


        Rupture Index  Annual Rate
0                   0          0.0
1                   1          0.0
2                   2          0.0
3                   3          0.0
4                   4          0.0
...               ...          ...
411265         411265          0.0
411266         411266          0.0
411267         411267          0.0
411268         411268          0.0
411269         411269          0.0

[411270 rows x 2 columns]

participation rate for fault 'Masterton': 0.00015440158313140273 /yr


In [18]:
## Participation demos

In [19]:
# All the subections in the solution
section_rates = sol.rs_with_rupture_rates.groupby("section").agg('sum')["Annual Rate"]
print("section participation rates")
print(section_rates)

section participation rates
section
0.0       0.000000
1.0       0.000000
2.0       0.000000
3.0       0.000049
4.0       0.000049
            ...   
2320.0    0.000000
2321.0    0.000007
2322.0    0.000007
2323.0    0.000000
2324.0    0.000000
Name: Annual Rate, Length: 2325, dtype: float32


In [20]:
df0 = sol.fault_sections_with_rupture_rates[["Annual Rate", "ParentName"]]
df1 = df0.groupby("ParentName").agg("sum")["Annual Rate"]
df1.filter(like=FAULT_NAME)

ParentName
Masterton    0.000176
Name: Annual Rate, dtype: float32